In [ ]:
import pandas as pd
from datetime import datetime
from QuantConnect.Research import QuantBook
from QuantConnect import Resolution

qb = QuantBook()

EXTENDED_HOURS = True

# Add SPY minute data
spy = qb.AddEquity("SPY", Resolution.Minute, extendedMarketHours=EXTENDED_HOURS).Symbol

start = pd.Timestamp("2014-12-30 04:00", tz="America/New_York")
end   = pd.Timestamp("2025-06-03 20:00", tz="America/New_York")

hist = qb.History(spy, start, end, Resolution.DAILY)

if isinstance(hist.index, pd.MultiIndex):
    df = hist.loc[spy].reset_index()
else:
    df = hist.reset_index()

# Convert time to UTC if needed
if 'time' in df.columns:
    df['time'] = pd.to_datetime(df['time'], utc=True)#.dt.tz_convert('America/New_York')

#mask = (df['time'].dt.time >= time(9, 0)) & (df['time'].dt.time <= time(10, 0))
#df = df.loc[mask, ['time', 'open', 'high', 'low', 'close', 'volume']].copy()

# Print CSV text to output
csv_text = df.to_csv(index=False)
print(csv_text)

In [ ]:
# Add instruments for factors
USO    = qb.AddEquity("USO", Resolution.Daily).Symbol      # Oil ETF
GLD    = qb.AddEquity("GLD", Resolution.Daily).Symbol      # Gold ETF
EURUSD = qb.AddForex("EURUSD", Resolution.Daily).Symbol    # Euro / USD
USDJPY = qb.AddForex("USDJPY", Resolution.Daily).Symbol    # USD / JPY


In [ ]:
# List all symbols
symbols = [spy, USO, GLD, EURUSD, USDJPY]

# Pull daily history
hist = qb.History(symbols, start, end, Resolution.Daily)

# Extract close + volume to wide tables
prices = hist.close.unstack(0).sort_index()
volumes = hist.volume.unstack(0).sort_index() if "volume" in hist.columns else pd.DataFrame(index=prices.index)

# Rename columns to string
prices.columns = [str(c) for c in prices.columns]
volumes.columns = [str(c) for c in volumes.columns]

print("✅ Data pulled for:", list(prices.columns))
print(prices.tail(3))


In [ ]:
def to_ny_date(df):
    """Convert timestamps to NY trading date (00:00 of that date) and take last value per day."""
    idx = df.index
    if idx.tz is None:
        idx = idx.tz_localize("UTC")
    idx = idx.tz_convert("America/New_York").normalize()
    df2 = df.copy()
    df2.index = idx
    return df2.groupby(df2.index).last()

# Convert to NY-date index
prices_d  = to_ny_date(prices)
volumes_d = to_ny_date(volumes) if not volumes.empty else pd.DataFrame(index=prices_d.index)

# Reference calendar = SPY trading days
ref_dates = prices_d["SPY"].dropna().index

# Reindex to SPY calendar
prices_d  = prices_d.reindex(ref_dates)
volumes_d = volumes_d.reindex(ref_dates)

# Fill missing data for non-SPY assets
for c in ["USO", "GLD", "EURUSD", "USDJPY"]:
    if c in prices_d.columns:
        prices_d[c] = prices_d[c].ffill().bfill()

# Fill SPY volume if needed
if "SPY" in volumes_d.columns:
    volumes_d["SPY"] = volumes_d["SPY"].ffill().bfill()

print("Aligned rows:", len(prices_d))
print("NaNs by column:\n", prices_d[["SPY","USO","GLD","EURUSD","USDJPY"]].isna().sum())

In [ ]:
feat = pd.DataFrame(index=prices_d.index)

# === SPY Flows ===
spy_close = prices_d["SPY"]
spy_vol   = volumes_d["SPY"]

feat["flow_vol_z20"]       = (spy_vol - spy_vol.rolling(20).mean()) / spy_vol.rolling(20).std()
feat["flow_money_proxy5"]  = (spy_close * spy_vol).pct_change(5)

# === Commodities ===
feat["uso_ret5"] = prices_d["USO"].pct_change(5) if "USO" in prices_d.columns else np.nan
feat["gld_ret5"] = prices_d["GLD"].pct_change(5) if "GLD" in prices_d.columns else np.nan

# === FX ===
feat["eurusd_ret5"] = prices_d["EURUSD"].pct_change(5) if "EURUSD" in prices_d.columns else np.nan
feat["usdjpy_ret5"] = prices_d["USDJPY"].pct_change(5) if "USDJPY" in prices_d.columns else np.nan
feat["usd_strength"] = 0.5 * (-feat["eurusd_ret5"]) + 0.5 * (feat["usdjpy_ret5"])

# Drop initial rolling window
MAX_LOOKBACK = 20
feat = feat.iloc[MAX_LOOKBACK:].replace([np.inf, -np.inf], np.nan).dropna(how="any")

print("Features created:", feat.shape)
feat.head(3)


In [ ]:
import os

if os.path.exists("spy_ohlcv.csv"):
    print("✅ spy_ohlcv.csv already exists.")
else:
    try:
        df.to_csv("spy_ohlcv.csv", index=False)
        print("✅ wrote spy_ohlcv.csv from existing df:", df.shape)
    except NameError:
        hist_spy = qb.History(spy, start, end, Resolution.Daily)
        if isinstance(hist_spy.index, pd.MultiIndex):
            df1_tmp = hist_spy.loc[spy].reset_index()
        else:
            df1_tmp = hist_spy.reset_index()
        # Normalize time column to UTC tz-aware
        if 'time' in df1_tmp.columns:
            df1_tmp['time'] = pd.to_datetime(df1_tmp['time'], utc=True)
        # Keep standard columns
        keep = [c for c in ['time','open','high','low','close','volume'] if c in df1_tmp.columns]
        df1_tmp = df1_tmp[keep].copy()
        df1_tmp.to_csv("spy_ohlcv.csv", index=False)
        print("✅ rebuilt & wrote spy_ohlcv.csv:", df1_tmp.shape)

df1_check = pd.read_csv("spy_ohlcv.csv", parse_dates=["time"])
print("spy_ohlcv.csv head:\n", df1_check.head(3))

In [ ]:
df1 = pd.read_csv("spy_ohlcv.csv", parse_dates=["time"])

# Convert SPY timestamps -> NY date (no tz)
t1 = pd.to_datetime(df1["time"], utc=True)
df1["ny_date"] = t1.dt.tz_convert("America/New_York").dt.date

# Right side: your features 'feat' (index is NY-date-like)
feat_out = feat.copy()

idx = feat_out.index
# If index is tz-aware, convert to NY; else just coerce to datetime then .date
if getattr(idx, "tz", None) is not None:
    ny_dates = idx.tz_convert("America/New_York").date
else:
    ny_dates = pd.to_datetime(idx).date

feat_out = feat_out.reset_index(drop=True)
feat_out["ny_date"] = ny_dates

# Merge on the DATE key
merged = pd.merge(df1, feat_out, on="ny_date", how="inner").drop(columns=["ny_date"])

# Save & preview
merged.to_csv("spy_features_full.csv", index=False)

In [ ]:
from QuantConnect import Resolution

try:
    RSP = qb.AddEquity("RSP", Resolution.Daily).Symbol
    rsp_hist = qb.History(RSP, start, end, Resolution.Daily)

    # Convert to wide close series
    if isinstance(rsp_hist.index, pd.MultiIndex):
        rsp_close_raw = rsp_hist.close.unstack(0)["RSP"]
    else:
        rsp_close_raw = rsp_hist.set_index("time")["close"]

    # Reuse your to_ny_date() helper from earlier cell
    rsp_close_d = to_ny_date(rsp_close_raw.to_frame(name="RSP"))["RSP"]

    # Align to SPY trading calendar and fill around holidays
    rsp_close_d = rsp_close_d.reindex(prices_d.index).ffill().bfill()

    print("✅ RSP aligned:", rsp_close_d.index.min(), "→", rsp_close_d.index.max(), "| NaNs:", rsp_close_d.isna().sum())
    has_rsp = True
except Exception as e:
    print("⚠️ RSP not available, skipping breadth proxy. Reason:", e)
    has_rsp = False


In [ ]:
# === Breadth proxy: equal-weight outperforming cap-weight (RSP vs SPY) ===
if has_rsp:
    spy_close = prices_d["SPY"]

    rsp_ret5 = rsp_close_d.pct_change(5)
    spy_ret5 = spy_close.pct_change(5)

    feat["breadth_proxy"] = (rsp_ret5 - spy_ret5)

    # Clean only new column's initial lookback; keep prior cleaning
    feat = feat.replace([np.inf, -np.inf], np.nan)

    feat = feat.dropna(subset=["breadth_proxy"])

    print("✅ Added breadth_proxy. Feature shape now:", feat.shape)
else:
    print("↩️ Proceeding without breadth_proxy. (Everything else unchanged.)")

In [ ]:
# === Save updated features ===
feat.to_csv("spy_features_step2.csv")
print("Saved spy_features_step2.csv:", feat.shape)

df1 = pd.read_csv("spy_ohlcv.csv", parse_dates=["time"])
t1  = pd.to_datetime(df1["time"], utc=True)
df1["ny_date"] = t1.dt.tz_convert("America/New_York").dt.date

# Build same NY-date key for features
feat_out = feat.copy()
idx = feat_out.index
if getattr(idx, "tz", None) is not None:
    ny_dates = idx.tz_convert("America/New_York").date
else:
    ny_dates = pd.to_datetime(idx).date
feat_out = feat_out.reset_index(drop=True)
feat_out["ny_date"] = ny_dates

merged = pd.merge(df1, feat_out, on="ny_date", how="inner").drop(columns=["ny_date"])

out_name = "spy_features_full_v2.csv" if "breadth_proxy" in feat.columns else "spy_features_full.csv"
merged.to_csv(out_name, index=False)

print(f"Final merged file: {out_name} | shape:", merged.shape)
print("Columns:", list(merged.columns))

print(merged)


In [ ]:
feat.to_csv("spy_features_step2.csv")
print("Saved spy_features_step2.csv:", feat.shape)

df1 = pd.read_csv("spy_ohlcv.csv", parse_dates=["time"])
t1  = pd.to_datetime(df1["time"], utc=True)
df1["ny_date"] = t1.dt.tz_convert("America/New_York").dt.date

# Build same NY-date key for features
feat_out = feat.copy()
idx = feat_out.index
if getattr(idx, "tz", None) is not None:
    ny_dates = idx.tz_convert("America/New_York").date
else:
    ny_dates = pd.to_datetime(idx).date
feat_out = feat_out.reset_index(drop=True)
feat_out["ny_date"] = ny_dates

# Merge everything
merged = pd.merge(df1, feat_out, on="ny_date", how="inner").drop(columns=["ny_date"])

# === Print full CSV to output ===
csv_text = merged.to_csv(index=False)
print(csv_text)

